### 8. Рекуррентные нейронные сети RNN LSTM GRU

#### Задание<br>
На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара

1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN  и (RNN -> CNN)
4. сдлать выводы что получилось лучше

Развернем окружение

In [2]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=9c881658920ca02f80abd88e745cd3dc76f1fa3c02ac77fcfb60ca628ac82967
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [5]:
!pip install stop_words

  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32896 sha256=dac187aa92a037f060bbd3ae988fa887e99dfb1dea20a4accc0af157e885ea26
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words


In [3]:
import numpy as np
import pandas as pd

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from pymorphy2 import MorphAnalyzer

import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, Flatten, SimpleRNN, LSTM, GRU, Reshape
from keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy

В работе будут применяться данные по датасету "Отзывы". Посмотрим на данные

In [6]:
from google.colab import files

In [7]:
uploaded = files.upload()

Saving Отзывы.xls to Отзывы.xls


In [8]:
df = pd.read_excel('Отзывы.xls')
df.head(10)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,У меня работает все четко. В отличии от банком...,2017-08-14
9,5,Очень все хорошо👍,2017-08-14


In [9]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

In [10]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Подготовим данные

In [11]:
stopwordslist = stopwords.words("russian")
ptrn = r'[^a-zA-Zа-яА-Я0-9]'

morpher = MorphAnalyzer()

def words_only(text):
    text=str(text)
    return text.lower()

def remove_punkt(text):
    return re.sub(ptrn, ' ', text)

def to_token(text):
    return nltk.tokenize.word_tokenize(text)

def remove_stopwords(text):
    text_list = [w for w in text if w not in stopwordslist]
    return ' '.join(word for word in text_list)

def morphe_text(text):
    text = [morpher.parse(word)[0].normal_form for word in text.split() if word not in stopwordslist]
    return " ".join(text)

def normalize_text(text):
    text = words_only(text)
    text = remove_punkt(text)
    text = to_token(text)
    text = remove_stopwords(text)
    text = morphe_text(text)
    return text

In [12]:
df['normalized_content'] = df['Content'].apply(normalize_text)
df.head()

,Rating,Content,Date,normalized_content
0,5,It just works!,2017-08-14,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложение минус хотеть слишко...
2,5,Отлично все,2017-08-14,отлично
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать 1 работа антивирус далёкий нику...
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро


In [14]:
train_corpus = " ".join(df['normalized_content'])
train_tokens = word_tokenize(train_corpus)
train_tokens_filtered = [word for word in train_tokens if word.isalnum()]

In [16]:
max_words = 20000
max_len = 150
num_classes = 5

epochs = 20
batch_size = 512
print_batch_n = 100

In [17]:
dist = FreqDist(train_tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
voc = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [19]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in voc:
            result.append(voc[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

data_train = np.asarray(
    [text_to_sequence(text, max_len) for text in df['normalized_content']],
    dtype=np.int32)

Сформируем выборки train/test

In [21]:
X_train, X_test, y_train, y_test = train_test_split(data_train, df.Rating, test_size=0.3, random_state=1)

le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

#### Обучим сеть CNN

In [22]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(GlobalMaxPool1D())
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 14s 138ms/step - loss: 1.2323 - accuracy: 0.6739 - val_loss: 1.1872 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 3s 135ms/step - loss: 0.9353 - accuracy: 0.7390 - val_loss: 0.9199 - val_accuracy: 0.7567
Epoch 3/5
23/23 [==============================] - 5s 216ms/step - loss: 0.7768 - accuracy: 0.7670 - val_loss: 0.8414 - val_accuracy: 0.7615
Epoch 4/5
23/23 [==============================] - 3s 113ms/step - loss: 0.6958 - accuracy: 0.7800 - val_loss: 0.7850 - val_accuracy: 0.7622
Epoch 5/5
23/23 [==============================] - 2s 83ms/step - loss: 0.6415 - accuracy: 0.7866 - val_loss: 0.7533 - val_accuracy: 0.7688


In [23]:
# помотрим метрики
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = pd.DataFrame(columns=['model', 'Test score', 'Test accuracy'])

metrics_df = metrics_df.append({
    'model': 'CNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 0s 19ms/step - loss: 0.7917 - accuracy: 0.7444
Test score: 0.7917404770851135
Test accuracy: 0.7444337010383606


<ipython-input-23-c78428c05ae6>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


#### Обучим сеть RNN

In [24]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 12s 418ms/step - loss: 1.4360 - accuracy: 0.6508 - val_loss: 1.0892 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 8s 365ms/step - loss: 0.8880 - accuracy: 0.7415 - val_loss: 0.7396 - val_accuracy: 0.7649
Epoch 3/5
23/23 [==============================] - 10s 420ms/step - loss: 0.6749 - accuracy: 0.7752 - val_loss: 0.6809 - val_accuracy: 0.7784
Epoch 4/5
23/23 [==============================] - 9s 398ms/step - loss: 0.6110 - accuracy: 0.7911 - val_loss: 0.6748 - val_accuracy: 0.7784
Epoch 5/5
23/23 [==============================] - 7s 298ms/step - loss: 0.5710 - accuracy: 0.8067 - val_loss: 0.6854 - val_accuracy: 0.7712


In [25]:
# посмотрим метрики
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'SimpleRNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 0s 36ms/step - loss: 0.7078 - accuracy: 0.7485
Test score: 0.7077803611755371
Test accuracy: 0.7484672665596008


<ipython-input-25-5f71de3ce575>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


#### Обучим сеть LSTM

In [26]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 25s 708ms/step - loss: 1.5599 - accuracy: 0.6739 - val_loss: 1.4315 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 13s 563ms/step - loss: 1.1330 - accuracy: 0.7030 - val_loss: 0.8583 - val_accuracy: 0.7155
Epoch 3/5
23/23 [==============================] - 12s 521ms/step - loss: 0.7621 - accuracy: 0.7047 - val_loss: 0.7090 - val_accuracy: 0.7504
Epoch 4/5
23/23 [==============================] - 11s 491ms/step - loss: 0.6658 - accuracy: 0.7669 - val_loss: 0.6823 - val_accuracy: 0.7812
Epoch 5/5
23/23 [==============================] - 10s 428ms/step - loss: 0.6234 - accuracy: 0.7851 - val_loss: 0.6773 - val_accuracy: 0.7836


In [27]:
# посмотрим метрики
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'LSTM',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 37ms/step - loss: 0.7050 - accuracy: 0.7649
Test score: 0.705035924911499
Test accuracy: 0.764924168586731


<ipython-input-27-66a0c3794823>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


#### Обучим сеть GRU

In [28]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 29s 919ms/step - loss: 1.4913 - accuracy: 0.6843 - val_loss: 1.2032 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 17s 682ms/step - loss: 0.9505 - accuracy: 0.7030 - val_loss: 0.8182 - val_accuracy: 0.7155
Epoch 3/5
23/23 [==============================] - 12s 479ms/step - loss: 0.7463 - accuracy: 0.7110 - val_loss: 0.7133 - val_accuracy: 0.7743
Epoch 4/5
23/23 [==============================] - 12s 514ms/step - loss: 0.6488 - accuracy: 0.7810 - val_loss: 0.6817 - val_accuracy: 0.7774
Epoch 5/5
23/23 [==============================] - 12s 543ms/step - loss: 0.5999 - accuracy: 0.7918 - val_loss: 0.6795 - val_accuracy: 0.7784


In [29]:
# посмотрим метрики
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'GRU',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 37ms/step - loss: 0.7088 - accuracy: 0.7583
Test score: 0.7087770104408264
Test accuracy: 0.7583091259002686


<ipython-input-29-355e340f78bc>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


#### Обучим комбинацию сетей CNN / RNN

In [30]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 9s 316ms/step - loss: 1.0619 - accuracy: 0.6833 - val_loss: 0.8402 - val_accuracy: 0.7179
Epoch 2/5
23/23 [==============================] - 7s 252ms/step - loss: 0.8122 - accuracy: 0.7302 - val_loss: 0.7401 - val_accuracy: 0.7549
Epoch 3/5
23/23 [==============================] - 5s 215ms/step - loss: 0.7047 - accuracy: 0.7624 - val_loss: 0.6730 - val_accuracy: 0.7674
Epoch 4/5
23/23 [==============================] - 7s 289ms/step - loss: 0.6524 - accuracy: 0.7723 - val_loss: 0.6788 - val_accuracy: 0.7618
Epoch 5/5
23/23 [==============================] - 5s 214ms/step - loss: 0.6130 - accuracy: 0.7897 - val_loss: 0.6631 - val_accuracy: 0.7722


In [31]:
# посмотрим метрики
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'CNN / RNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 0s 17ms/step - loss: 0.6914 - accuracy: 0.7556
Test score: 0.691429078578949
Test accuracy: 0.7555662989616394


<ipython-input-31-1b7f9669fad0>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


#### Обучим комбинированную сеть CNN / LSTM

In [32]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 6s 147ms/step - loss: 1.4442 - accuracy: 0.6154 - val_loss: 1.2952 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 3s 117ms/step - loss: 1.1438 - accuracy: 0.7033 - val_loss: 0.9463 - val_accuracy: 0.7152
Epoch 3/5
23/23 [==============================] - 2s 100ms/step - loss: 0.8607 - accuracy: 0.7305 - val_loss: 0.7786 - val_accuracy: 0.7473
Epoch 4/5
23/23 [==============================] - 4s 183ms/step - loss: 0.7135 - accuracy: 0.7637 - val_loss: 0.7228 - val_accuracy: 0.7563
Epoch 5/5
23/23 [==============================] - 2s 82ms/step - loss: 0.6406 - accuracy: 0.7793 - val_loss: 0.7113 - val_accuracy: 0.7601


In [33]:
# посмотрим на метрики
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'CNN / LSTM',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 0s 20ms/step - loss: 0.7326 - accuracy: 0.7457
Test score: 0.7326353192329407
Test accuracy: 0.7457244396209717


<ipython-input-33-cc91ca7c24b4>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


#### Обучим комбинированную сеть CNN / GRU

In [34]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 9s 186ms/step - loss: 1.1692 - accuracy: 0.6881 - val_loss: 0.8125 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 6s 251ms/step - loss: 0.7829 - accuracy: 0.7093 - val_loss: 0.7146 - val_accuracy: 0.7497
Epoch 3/5
23/23 [==============================] - 2s 107ms/step - loss: 0.6874 - accuracy: 0.7637 - val_loss: 0.6761 - val_accuracy: 0.7660
Epoch 4/5
23/23 [==============================] - 2s 88ms/step - loss: 0.6149 - accuracy: 0.7820 - val_loss: 0.6639 - val_accuracy: 0.7684
Epoch 5/5
23/23 [==============================] - 2s 102ms/step - loss: 0.5568 - accuracy: 0.8026 - val_loss: 0.6800 - val_accuracy: 0.7674


In [35]:
# посмотрим на метрики
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'CNN / GRU',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 0s 21ms/step - loss: 0.7112 - accuracy: 0.7523
Test score: 0.71119225025177
Test accuracy: 0.7523394823074341


<ipython-input-35-5238e8a3e0fe>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


#### Обучим комбинированную сеть RNN / CNN, поменяв очередность слоев

In [36]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 13s 444ms/step - loss: 1.1844 - accuracy: 0.6837 - val_loss: 0.8621 - val_accuracy: 0.7217
Epoch 2/5
23/23 [==============================] - 11s 437ms/step - loss: 0.8121 - accuracy: 0.7259 - val_loss: 0.7182 - val_accuracy: 0.7535
Epoch 3/5
23/23 [==============================] - 12s 535ms/step - loss: 0.6594 - accuracy: 0.7675 - val_loss: 0.6850 - val_accuracy: 0.7722
Epoch 4/5
23/23 [==============================] - 11s 494ms/step - loss: 0.5666 - accuracy: 0.7929 - val_loss: 0.7129 - val_accuracy: 0.7688
Epoch 5/5
23/23 [==============================] - 10s 416ms/step - loss: 0.4877 - accuracy: 0.8237 - val_loss: 0.7656 - val_accuracy: 0.7722


In [37]:
#посмотрим на метрики
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'RNN / CNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 0s 25ms/step - loss: 0.7738 - accuracy: 0.7594
Test score: 0.7738342881202698
Test accuracy: 0.7594385147094727


<ipython-input-37-5b638bbb83e8>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


#### Обучим комбинированную сеть LSTM / CNN

In [38]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(LSTM(128, return_sequences=True))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 26s 804ms/step - loss: 1.2122 - accuracy: 0.6827 - val_loss: 0.9542 - val_accuracy: 0.7159
Epoch 2/5
23/23 [==============================] - 13s 593ms/step - loss: 0.8648 - accuracy: 0.7261 - val_loss: 0.7433 - val_accuracy: 0.7542
Epoch 3/5
23/23 [==============================] - 18s 809ms/step - loss: 0.6859 - accuracy: 0.7640 - val_loss: 0.6643 - val_accuracy: 0.7643
Epoch 4/5
23/23 [==============================] - 14s 604ms/step - loss: 0.5963 - accuracy: 0.7880 - val_loss: 0.6509 - val_accuracy: 0.7681
Epoch 5/5
23/23 [==============================] - 14s 632ms/step - loss: 0.5380 - accuracy: 0.8073 - val_loss: 0.6657 - val_accuracy: 0.7743


In [39]:
#посмотрим на метрики
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'LSTM / CNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 67ms/step - loss: 0.7014 - accuracy: 0.7535
Test score: 0.7013879418373108
Test accuracy: 0.7534688711166382


<ipython-input-39-66618bcdc427>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


#### Обучим комбинированную сеть GRU / CNN


In [47]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(GRU(128, return_sequences=True))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 20s 631ms/step - loss: 1.2498 - accuracy: 0.6798 - val_loss: 0.9358 - val_accuracy: 0.7249
Epoch 2/5
23/23 [==============================] - 12s 516ms/step - loss: 0.8758 - accuracy: 0.7273 - val_loss: 0.7775 - val_accuracy: 0.7515
Epoch 3/5
23/23 [==============================] - 14s 609ms/step - loss: 0.7047 - accuracy: 0.7629 - val_loss: 0.6713 - val_accuracy: 0.7719
Epoch 4/5
23/23 [==============================] - 12s 542ms/step - loss: 0.5924 - accuracy: 0.7873 - val_loss: 0.6672 - val_accuracy: 0.7722
Epoch 5/5
23/23 [==============================] - 13s 551ms/step - loss: 0.5229 - accuracy: 0.8057 - val_loss: 0.7089 - val_accuracy: 0.7729


In [48]:
#посмотрим на метрики
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'GRU / CNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 38ms/step - loss: 0.7322 - accuracy: 0.7604
Test score: 0.7322380542755127
Test accuracy: 0.7604065537452698


<ipython-input-48-10135b01d5b0>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


#### Объединим полученные данные в таблицу

In [49]:
metrics_df

,model,Test score,Test accuracy
0,CNN,0.791740,0.744434
1,SimpleRNN,0.707780,0.748467
2,LSTM,0.705036,0.764924
3,GRU,0.708777,0.758309
4,CNN / RNN,0.691429,0.755566
5,CNN / LSTM,0.732635,0.745724
6,CNN / GRU,0.711192,0.752339
7,RNN / CNN,0.773834,0.759439
8,LSTM / CNN,0.701388,0.753469
9,GRU / CNN,0.732238,0.760407


Выводы: по результатам тренировки **сверточная сеть (CNN)** показывает лучший сбалансированный результат в целом и по **f1_score** в частности, тогда как лучшее **accuracy** у комбинированной архитектуры **GRU/CNN**. Результаты всех комбинированных сетей схожи, возможно имеет место небольшое переобучение, где нужно более тчательно подготавливать данные и подбирать отдельные параметры обучения под каждую архитектуру.